# Week 11b - Identify Halifax's Twitter influencers _without_ Netlytic
*© 2021 Colin Conrad*

Welcome to Week 11 of INFO 6270! Last week we covered SQL databases. This week we are going to go back to core Python skills and conduct social media data analysis. Though it is no longer hyped, social media analysis remains an important and interesting skill for data scientists to develop. 

**Note:** This is an alternative lab for people who did not have access to a Twitter account or were opposed to having one (a totally acceptable and understandable life choice).

**This week, we will achieve the following objectives:**
- Get Twitter data into Python
- Conduct basic natural language processing (NLP)
- Visualize your word frequencies
- Identify Twitter influencers through network analysis

# Case: Affinio
[Affinio](https://www.affinio.com/) is a Halifax-based social media analytics Software as a Service (SaaS) company which offers the Affinio marketing insight engine. Affinio uses social media data to generate insights about a company's existing customers by identifying information about their activities, interests and opinions and return the insight generated to the company's customer relationship management software (CRM). This helps companies better understand their customer needs.

Though not always advertised, the main feature that makes the Affinio algorithm work is its network analysis features. In this lab, we will investigate social media text and network features that similarly generate insights, in our case, about Halifax.

# Objective 1: Get Twitter data into Python
In many ways, Twitter is a great resource for academic researchers because it offers access to large amounts of social data for free. In the recent past, this meant that students would be able able to easily register for a Twitter Developer account using their Twitter login credentials and have instant access to data. However, [starting 2018](https://blog.twitter.com/developer/en_us/topics/tools/2018/new-developer-requirements-to-protect-our-platform.html) Twitter changed its developer account process in an effort to limit spam. Students can still access developer credentials, however you must be manually vetted by Twitter, which makes it difficult to do the lab using the Twitter API. You can still do this if given enough time however, and if interested, consider doing your final project by accessing Twitter data directly in Python using the [Tweepy](https://www.tweepy.org/) library.

Instead, we will use a tool called [Netlytic](https://netlytic.org/) to help us access the data that we are looking for. Netlytic is a free-to-use tool maintained by Dr. Anatoliy Gruzd and Philip Mai at Ryerson University and is designed to help academic-types such as us. A bit of trivia, Anatoliy and Philip used to work at the School of Information Management at Dalhousie. This section will give you a step-by-step guide for conducting your own analysis using Netlytic. I also similarly downloaded a dataset which retrieved the `#Halifax` keyword for reference.

## Step 1: Load your CSV files
You have been provided with two data sets from Netlytic. Let's start by exploring the first contained in `hfx-tweets.csv`. This consists of a series of tweets from 2020 from the Halifax area, collected using the `halifax` keyword. Let's look at what the CSV file has for us.

In [ ]:
import csv

with open('data/w11_hfx-tweets.csv', newline='') as tweetsfile:
    reader = csv.reader(tweetsfile, delimiter=',')
    for row in reader:
        print(row)

This code might trigger an error. If so, you probably encountered this error because of an text encoding problem. One challenge of working with Twitter data is that they contain emojis, which are not part of the default encoding used by Python's CSV library. Instead, we have to identify the proper encoding type, which in this case is `utf-8`. Try executing this code instead.

In [ ]:
import csv

tweets = []

with open('data/w11_hfx-tweets.csv', newline='', encoding='utf-8') as tweetsfile: # changed to support utf-8 encoding
    reader = csv.reader(tweetsfile, delimiter=',')
    for row in reader:
        tweets.append(row)
        
print(tweets)

Voila! You have extracted the Tweets from the CSV file. You can also investigate the data to recognize that these are expressed as a series of lists. We will come back to these soon.

## *Challenge Question 1 (1 point)*
The second file is called `w11_coronavirus-tweets.csv`. Using the code above as a foundation, import this CSV file and print the number of tweets contained in it. You should be able to use the `for` loop to detect the number of tweets contained.

In [ ]:
# insert your code here

# Objective 2: Conduct basic natural language processing (NLP)
Now that we have the social media data in an accessible format, we can start conducting analysis on the text data. Within the broad subject of data science, there is a sub-discipline called *natural language processing* (NLP) which investigates computational analysis related to textual data. NLP is an interesting interdisciplineary field that draws on more than just machine learning; it also uses techniques from linguistics (such as semantic structure and grammar) and information management (such as ontologies)! We will not dive too deeply into NLP theory in this course, though that can't stop us from using it to achieve a goal.

To stary, we can investigate some of Netlytic's features. Using Netlytic, we can process counts of keywords in our data set. Let's illustrate the steps taken to generate the `w11_hfx-freqs.csv` file contained in your data folder.

## Conducting NLP with Python ourselves, without Netlytic
By now you should know enough to not be too impressed by this feature of Netlytic. Generating word counts such as this using Python is relatively simple. Think about the logic for such a function:
- Create a dictionary for terms
- Loop through each term from every tweet
- If the term does not exist in the dictionary, add it
- If the term does exist in the dictionary, increase its count by one
- When the loop is finished, export it to a CSV file

We could write this function ourselves if we needed to. Still, this would have a few problems. One such problem is that the most common words in English are words such as `the`, `a`, `of`, `and`, `but`. Note that these did not make it into Netlytic's word cloud. In NLP these are often called *stopwords* and are filtered out of a keyword analysis. It would be a pain to filter stopwords, as well as prepare our functions for the nuances of natural language.

The secret to making NLP work in Python is to use the right tools for the job. Python has an excellent library called `NLTK`: the [Natural Language ToolKit](https://www.nltk.org/). NLTK is among Python's most powerful libraries and is (in my opinion) one of the reasons why Python is a preferred tool for data science. NLTK contains methods for conducting tasks such as: 
- sentence grammar computation 
- cluster analysis 
- keyword tagging
- translation

By using NLTK we can incorporate all of these features (and more!) into our analysis. Let's start with something simple though. Let's start by generating a list of keywords similarly to how Netlytic worked. The first step is to transform our tweet sentences into words. In NLP, these are called word tokens. The code below selects uses NLTK's built-in `TweetTokenizer` to identify tokens which are specific to tweets and transform a tweet into said tokens.

In [ ]:
import nltk # import the library
from nltk.tokenize import TweetTokenizer # import TweetTokenizer

tokenizer = TweetTokenizer() # import the tokenizer object

tweet_tokens = tokenizer.tokenize(tweets[4][6]) # fourth tweet in our list, 6th column (the tweet text)

print(tweet_tokens)

Pretty cool! Similarly, if we would like to combine all of the tokens from all of the tweets, we could simply create a master string which contains all of the tweet content. Let's combine all of the tweets into one string. 

In [ ]:
combined_tweets_text = "" # create a placeholder string

for t in tweets:
    combined_tweets_text = combined_tweets_text + " " + t[6] # append the tweet content

With the tweets combined we can again run the tokenizer. This will give us a series of keywords.

In [ ]:
tweets_tokens = tokenizer.tokenize(combined_tweets_text) # fourth tweet in our list, 6th column (the tweet text)

print(tweets_tokens)

We can now conduct analysis on the tweet content as we would any other list data! For instance, if we wanted to pick out the number of times the word "Bedford" appeared in our tweets, we could write a simple counting script, such as the one below.

In [ ]:
# a simple Bedford counter script

num_bedford = 0

for word in tweets_tokens:
    if word == "Bedford":
        num_bedford += 1

print(num_bedford)

In addition to regular analysis, NLTK comes with some functions to make things easier. For instance, we may wish to analyse the frequency distribution (i.e. the number of times a word appears) of all of the terms in the tweets. For this, we can use the `FreqDist` function. The code below unpacks the 100 most common words in our tweets, just like Netlytic!

In [ ]:
fdist1 = nltk.FreqDist(tweets_tokens) # call FreqDist on the tweets_tokens

print(fdist1.most_common(100)) # print the 100 most common terms

## How to clean your tweet stopwords
This is good, though we are still not quite there. The top of our list is dominated by words such as `is` and `a` which are not very informative. In NLP these are referred to as *stop words*, the sort of words that are very common but minimally informative. In order to replicate what Netlytic did, we would need to remove the stop words. There is no magic formula for this one.

This said, NLTK does have some useful resources. NLTK contains more than functions, it also contains many corpora which can be useful to doing natural language tasks. One of these is the `stopwords` corpus, which contains a series of stopwords. Using Python we can create a loop which filters based on whether a keyword matches a word from that corpus.

In order to download an NLTK corpus, you need to use the `nltk.download` function. Execute the code below. An interface will generate.

In [ ]:
nltk.download()

Using the NLTK interface, navigate to the `Corpora` tab and search for the `stopwords` corpus. Download this, similarly to the picture below.

![alt text](img/7-NLTK.png "Download the stopwords corpus")

When downloaded, we can take a look at the content. Execute the line below once it is successfully downloaded to see the stopwords data. For fun, I have set this to demonstrate Arabic stopwords. Consider changing it to `english` to see the English version.

In [ ]:
from nltk.corpus import stopwords # import the stopwords corpus

print(stopwords.words('arabic')) # consider chaning me to English, German, Russian or Greek

We can now use the stopwords to clean our Twitter sample to remove the stopwords. Execute the code below to see the `cleaned` data set. 

In [ ]:
from nltk.corpus import stopwords
    
cleaned_tokens = [] # placeholder for the new tokens
for t in tweets_tokens:
    if t not in stopwords.words('english'): # check to see if it is in the stopwords corpus. If not, add it.
        cleaned_tokens.append(t) 

print(cleaned_tokens)

Finally, let's again create a frequency distribution to see the most common keywords. This is still not perfect, but this is progress!

In [ ]:
fdist2 = nltk.FreqDist(cleaned_tokens)

print(fdist2.most_common(100))

## *Challenge Question 2 (1 point)*
There are two major ways that the stopword solution above can be improved:
1. Some tags are essential identical but are considered different because they contain upper case letters;
2. We have identified stray characters (e.g. `.` and `,`) as keywords.

Modify the script to exclude these characters and to overcoming the casing issue. **Hint** you may wish to consider python's `.lower()` [method to convert all keywords to lowercase](https://www.w3schools.com/python/ref_string_lower.asp).

In [ ]:
# insert your code here

## *Challenge Question 3 (1 point)*
In addition to the Halifax script, you have also likely generated your own tweets generated by a different set of Twitter users. Instead of analyzing Tweets, analyze the most frequent terms contained in the users' Twitter `user_bio`. Your code should do the following:
- Import your csv data (or use the one on COVID-19 provided);
- Create a list of strings that contain profile descriptions instead of tweets
- Filter the stopwords
- Print your frequencies

You do *not* have to filter for repeated profiles in this question. 

### Import your data file

In [ ]:
# insert your code here

### Combine your profiles

In [ ]:
# insert your code here

### Create a processing script

In [ ]:
# insert your code here

# Objective 3: Visualize your word frequencies
So far in this course we have managed data by observing raw datasets. These are only so good in the grand scheme of things-- it is now time to take our first dive into visualization! Though we will explore some of these concepts in more detail later, I would like to now introduce `matplotlib`, one of Python's main visualization libraries. This library allows us to generate graphics relatively quickly using a few lines of code.

Let's start by visualizing word rank vs frequency. As you recall, `fdist2` consists of our frequency distributions from the Halifax tweets dataset. 

In [ ]:
fdist2

In their book *Mining the Social Web* (2019) Russell & Klassen point out that a `log scale graph` is helpful for these types of datasets. This type of graph shows the frequency and word rank in log space. This is helpful because some of our terms are much more common than others. Execute the code below to produce a graph of a log space visualization.

In [ ]:
import matplotlib.pyplot as plt # import matplotlib

# the line below for telling matplotlib not to pop out and generate graphs in Jupyter
%matplotlib inline 

counts = sorted(fdist2.values(), reverse=True) # sorted counts from fdist2

plt.loglog(counts) # plot a log space chart
plt.ylabel("Freq") # y axis label
plt.xlabel("Word Rank") # x axis label

plt.figure # call the plot

In addition, we could express word count frequencies as histograms-- graphs used to compare frequencies in statistics. For this, we could consider taking only the 100 most common values. With these we can investigate the distribution of the top keywords (Russell & Klassen, 2019).

In [ ]:
counts = fdist2.most_common(100) # take the 100 most common words

values = [] # this is a list
for c in counts: # get only the values, not the words themselves
    values.append(c[1])

plt.hist(values) # the data to be plotted
plt.title("Top Keywords") # titles and labels, as before
plt.ylabel("Number of items in bin")
plt.xlabel("Bins (frequency)")

plt.figure # call the plot

We don't need to know too much about what is happening on the backend yet, though this is a useful demonstration of how to visualize our findings.

## *Challenge Question 4 (0.5 point)*
Similarly to the log scale frquency diagram above, create a log-frequency distribution of your tweets data. 

In [ ]:
# insert your code here

## *Challenge Question 5 (0.5 points)*
Modify the histogram code above to work with your data. Unlike the histogram data above, select remove the top 10 words from your analysis. **Hint** you can accomplish this by subsetting your `values` list itself.

In [ ]:
# insert your code here

# Objective 4: Go beyond Twitter
In addition to APIs, we can also use Python to retrieve and process regular web data. In Lab 4 we tried this using the `requests` module, we retrieved a series of unreadable HTML text. This is a painful way to scrape web data, and it would take us hours to get to the point where we were able to do analytics similar to this lab. It would be much easier to process this type of data if there was a more efficient library.

Fortunately, Python has `Beautiful Soup` which is designed exactly for this task. This library structures HTML data retrieved using requests in a way that is not only readable, but also manageable. For instance, if we wanted to retrieve the Open Library home page, we could execute the following code.

**Note: It is possible that the Beautiful Soup `bs4` library is not installed. If not, use `pip install bs4` before executing this code.**

In [ ]:
import bs4 # import the Beautiful Soup library
res = requests.get('https://openlibrary.org/') # the target URL
res.raise_for_status() # checks for errors
librarySoup = bs4.BeautifulSoup(res.text, 'html.parser') # retrieve the text and format it as HTML
print(librarySoup) #print the HTML

Beautiful Soup has a few handy functions that greatly lightening our load when processing web data. We could save this HTML data by opening a file and saving the content of the retrieved website on our local computer. For instance, the following code retrieves Open Library's web page and saves the code on our local computer in the `data` folder.

In [ ]:
exampleFile = open('data/example.html', 'w', encoding='utf-8') # we need to explicitly state UTF8 encoding
exampleFile.write(str(librarySoup)) # writes the file
exampleFile.close() # closes the html file

Try opening the file using a code editor such as Notepad++. You will see that you have just copied Open Library's web page; this is to say, you **scraped** Open Library's web page. This example illustrates how computers access and process web data. Web scrapers also form the backbone of search engine technology and also the Open Internet Archive's software.

Web scrapers are ubiquitous, though they may not necessarily be legal in many circumstances. Many (or perhaps even most) web materials are copyrighted (e.g. many newspaper articles) and may not permit you accessing their data in this way. Fortunately the Open Internet Archive allows scholars to access their materials. Other sites may not be so generous.

### Retrieving specific web data
Using Beautiful Soup we can also access particular page elements. HTML documents consist of a series of elements which could include tags (e.g. `<div>`) as well as properties (e.g. the logo class `.logo`). Beautiful Soup helps us to navigate these elements so that we can retrieve the data that we want, rather than whole web pages.

This is better expressed using an example. If we wanted to retrieve data from specific elements from the Open Library web page, we can use the `select` method to retrieve that data. The following code retrieves only data which is contained in their `page-banner` class (usually reserved for important catch phrases).  

In [ ]:
res = requests.get('https://openlibrary.org/') # the target URL
res.raise_for_status() # checks for errors
librarySoup = bs4.BeautifulSoup(res.text, 'html.parser') # retrieve the text and format it as HTML
elems = librarySoup.select('.page-banner') # select only elements with the page-banner class
elems # print the data retrieved

Beautiful Soup detected two elements with this feature. The first was the donate button and the second was their catch phrase "Together, let's build an Open Library for the world". Beautiful Soup retrieved these in a list format, so we can retrieve the second of these elements using the following code.

In [ ]:
elems[1]

Beautiful soup's elements object also has a specific `getText()` method for retrieving only text. Using this we can retrieve the slogan from their web page. A picture of the exact element retrieved is provided for your reference.

In [ ]:
elems[1].getText()

Try this on your own with other web pages! How do you think you could use this skill to analyze text, similarly to social media data?

## Challenge Question 6 (1 point)
Using the Beautiful Soup library, retrieve and print the HTML data from `https://dal.ca`. You can modify the code we used to retrieve the Open Library page for this task.

In [ ]:
# insert your code here

## References

Russell, M. A. & Klassen, M. (2019). *Mining the social web, 3rd edition*. O'Reilly Press. 